In [9]:
import numpy as np 
import pandas as pd
import os

from tabula import read_pdf
from tabulate import tabulate

In [ ]:
# Define Dir with working raw datasets
os.getcwd()

pdf_dir = 'Raw Data/'
print(os.listdir(pdf_dir))

output_dir = 'Cleaned Data/'
print(os.listdir(output_dir))


In [28]:
# Test on specific states
tables_pg3 = read_pdf(os.path.join(pdf_dir, "URS_PuertoRico.pdf"), pages=3, multiple_tables=True)
df_total = pd.DataFrame(tables_pg3[0])

df_total.columns

df_total = df_total[['Utilization', 
                    'State Number',
                    'State Rate']][0:3] ## only select the first 3 rows


display(df_total)

,Utilization,State Number,State Rate
0,"Penetration Rate per 1,000 population","2,819",0.86
1,"Community Utilization per 1,000 population","1,666",0.51
2,"State Hospital Utilization per 1,000 population",421,0.13


In [34]:
# Define a function process_pdf to process raw pdf and save into a dataframe

def process_pdf(pdf_file, state_code): 

    ## Read table on page 3

    tables_pg3 = read_pdf(os.path.join(pdf_dir, pdf_file), pages=3, multiple_tables=True)
    
    ## Read tables[0] into a dataframe 
    df_total = pd.DataFrame(tables_pg3[0])
    df_total = df_total[['Utilization', 
                         'State Number',
                         'State Rate']][0:3] ## only select the first 3 rows

    ## Read tables from page 5 
    tables_pg5= read_pdf(os.path.join(pdf_dir,pdf_file), pages=5, multiple_tables=True )

    df_age = pd.DataFrame(tables_pg5[0])

    ## Only selecting specfic rows and columns that are associated with served patients & rates
    df_ageshort = df_age[["Unnamed: 0", "Unnamed: 1", "Unnamed: 5"]][4:13]

    ## Rename the df_ageshort dataframe by the column index
    df_2 = df_ageshort.rename(columns={df_ageshort.columns[0]: "Measure", 
                                       df_ageshort.columns[1]: "PatientsServed", 
                                       df_ageshort.columns[2]: "ServedRate"})



    ## Combine Total measure summary table with the age table 
    df_total = df_total.rename(columns= {df_total.columns[0]: "Measure", 
                                         df_total.columns[1]: "PatientsServed", 
                                         df_total.columns[2]: "ServedRate"})


    df_transformed = df_total.append(df_2)

    ## Add a column indicating the state 
    df_transformed['State'] = state_code

    return df_transformed




In [37]:
df_transformed.head(10)

df_transformed.to_excel()

,Measure,PatientsServed,ServedRate,State
0,"Penetration Rate per 1,000 population",510,4.86,USVirginIslands
1,"Community Utilization per 1,000 population",442,4.21,USVirginIslands
2,"State Hospital Utilization per 1,000 population",-,-,USVirginIslands
4,0-12,8,-,USVirginIslands
5,13-17,11,-,USVirginIslands
6,18-20,29,-,USVirginIslands
7,21-24,33,-,USVirginIslands
8,25-44,196,-,USVirginIslands
9,45-64,185,-,USVirginIslands
10,65-74,34,-,USVirginIslands


In [32]:
## Testing on individual pdf 
tables_pg5= read_pdf(os.path.join(pdf_dir, 'URS_Utah.pdf'), pages=5, multiple_tables=True)

df_age = pd.DataFrame(tables_pg5[0])

## Only selecting specfic rows and columns that are associated with served patients & rates
df_ageshort = df_age[["Unnamed: 0", "Unnamed: 1", "Unnamed: 5"]][4:13]

## Rename the df_ageshort dataframe by the column index
df_2 = df_ageshort.rename(columns={df_ageshort.columns[0]: "Measure", 
                                       df_ageshort.columns[1]: "PatientsServed", 
                                       df_ageshort.columns[2]: "ServedRate"})

display(df_2)

,Measure,PatientsServed,ServedRate
4,0-12,"12,205",18.4
5,13-17,"8,962",31.5
6,18-20,"2,861",18.8
7,21-24,"3,720",16.4
8,25-44,"19,744",20.7
9,45-64,"10,143",15.2
10,65-74,"1,778",7.3
11,75 and over,578,3.9
12,Age Not Available,-,-


In [35]:
# Check if the output dir exists, if not, creat a dir
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)

# Create an empty list to save all the state-specific dataframe
all_transformed_data = []
# Loop through all PDF files in the PDF dir
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"): 
        state_code = pdf_file.split('_')[1].split('.')[0] # extract the state code from the file name
        df_transformed = process_pdf(pdf_file, state_code)
        print(df_transformed.head(20))
        all_transformed_data.append(df_transformed)
    
    

# Combine all states into a single dataframe
combined_df = pd.concat(all_transformed_data, ignore_index=True)

display(combined_df)


/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/1843001140.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


                                            Measure PatientsServed ServedRate  \
0             Penetration Rate per 1,000 population        127,441      11.80   
1        Community Utilization per 1,000 population        127,441      11.80   
2   State Hospital Utilization per 1,000 population          2,726       0.25   
4                                              0-12          4,242        2.4   
5                                             13-17          5,942        7.7   
6                                             18-20          4,267        9.6   
7                                             21-24          9,301       16.0   
8                                             25-44         55,150       18.9   
9                                             45-64         42,105       15.4   
10                                            65-74          5,446        5.5   
11                                      75 and over            884        1.5   
12                          

/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/1843001140.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


                                            Measure PatientsServed ServedRate  \
0             Penetration Rate per 1,000 population         70,546      11.97   
1        Community Utilization per 1,000 population         69,258      11.75   
2   State Hospital Utilization per 1,000 population          6,124       1.04   
4                                              0-12          4,720        5.3   
5                                             13-17          8,479       21.9   
6                                             18-20          3,770       15.9   
7                                             21-24          4,573       14.6   
8                                             25-44         24,484       16.5   
9                                             45-64         18,532       12.1   
10                                            65-74          4,229        6.5   
11                                      75 and over          1,759        4.3   
12                          

/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/1843001140.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


...

                                            Measure PatientsServed ServedRate  \
0             Penetration Rate per 1,000 population         92,013      25.52   
1        Community Utilization per 1,000 population         91,024      25.25   
2   State Hospital Utilization per 1,000 population          1,152       0.32   
4                                              0-12         19,399       38.9   
5                                             13-17         18,015       78.2   
6                                             18-20          3,828       25.0   
7                                             21-24          4,021       21.5   
8                                             25-44         20,265       22.4   
9                                             45-64         20,063       20.5   
10                                            65-74          4,965       13.1   
11                                      75 and over          1,436        5.3   
12                          

/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/1843001140.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


                                            Measure PatientsServed ServedRate  \
0             Penetration Rate per 1,000 population         32,150      18.04   
1        Community Utilization per 1,000 population         30,262      16.98   
2   State Hospital Utilization per 1,000 population          1,399       0.79   
4                                              0-12          1,741        7.0   
5                                             13-17          2,252       20.7   
6                                             18-20          1,338       20.6   
7                                             21-24          1,900       21.5   
8                                             25-44         12,528       29.3   
9                                             45-64          9,926       20.9   
10                                            65-74          2,032        9.0   
11                                      75 and over            433        3.0   
12                          

/var/folders/55/27d1cbps3wdcwcpwg9hs8vg80000gn/T/ipykernel_62045/1843001140.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_transformed = df_total.append(df_2)


,Measure,PatientsServed,ServedRate,State
0,"Penetration Rate per 1,000 population","127,441",11.80,Georgia
1,"Community Utilization per 1,000 population","127,441",11.80,Georgia
2,"State Hospital Utilization per 1,000 population","2,726",0.25,Georgia
3,0-12,"4,242",2.4,Georgia
4,13-17,"5,942",7.7,Georgia
...,...,...,...,...
463,25-44,196,-,USVirginIslands
464,45-64,185,-,USVirginIslands
465,65-74,34,-,USVirginIslands
466,75 and over,14,-,USVirginIslands
